In [41]:
import glob
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re

In [42]:
optimal_plans_path = 'optimal_plans/'
folders = os.listdir(optimal_plans_path)
opt_dict = {fold: glob.glob(os.path.join(optimal_plans_path, fold, 'optimal_plans') + '/*.txt') 
            for fold in folders if ('problems' in fold)}

opt_dict.keys()

dict_keys(['clemens_problems_12_test', 'test_problems_18', 'clemens_problems_12', 'clemens_problems_18_test', 'test_problems_12', 'clemens_problems_18'])

In [43]:
results_dict = {folder: {method: glob.glob( os.path.join(folder, method) + '/plan*.txt' ) 
                         for method in os.listdir( os.path.join(folder) ) if 'DS_Store' not in method } 
                         for folder in os.listdir('./') if ('results' in folder) and os.path.isdir(folder)}

results_dict.keys()

dict_keys(['clemens_results_18', 'sas_results', 'test_results_12', 'clemens_results_12', 'test_results_18'])

In [44]:
for key in results_dict:
    if 'DeepCubeA' in key:
        continue

    if 'sas' not in key:
        key_in_opt_dict = key.replace('results', 'problems')
        if 'clemens' in key:
            key_in_opt_dict += '_test'
    else:
        # continue
        key_in_opt_dict = 'clemens_problems_18_test'

    for subkey in results_dict[key]:
        count = 0
        for item in results_dict[key][subkey]:

            if 'sas' not in key:
                item_basename = os.path.basename(item)
                item_name_opt = item_basename.replace('plan', 'plan_problem')
            else:
                item_basename = os.path.basename(item)
                problem_numbers = re.findall(r'(\d+)', item_basename)
                item_name_opt = f'plan_problem_{problem_numbers[0]}_{problem_numbers[1]}.txt'

            opt_item_path = opt_dict[key_in_opt_dict][0].split('plan_problem')[0] + item_name_opt

            with open(opt_item_path, 'r') as f:
                opt_content = f.read()
            
            with open(item, 'r') as f:
                content = f.read()

            item_cost = int(content.split('=')[1].split('(')[0].strip())
            opt_item_cost = re.search(r'(\d+)', opt_content.split('*')[0] ).group(1)
            if item_cost == int(opt_item_cost):
                count += 1
        print(key, subkey, len(results_dict[key][subkey]) , round(count/len(results_dict[key][subkey]) * 100,2) )
            


clemens_results_18 blind 65 100.0
clemens_results_18 merge-and-shrink 95 100.0
clemens_results_18 max-systematic-pdb 73 100.0
clemens_results_18 context_enhanced_additive 75 78.67
clemens_results_18 max-manual-pdb 78 100.0
clemens_results_18 goal_count 85 89.41
clemens_results_18 max 47 100.0
clemens_results_18 lm_cost 86 100.0
clemens_results_18 ff 113 79.65
clemens_results_18 causal_graph 75 80.0
sas_results blind 66 100.0
sas_results merge-and-shrink 90 100.0
sas_results max-systematic-pdb 120 100.0
sas_results context_enhanced_additive 77 75.32
sas_results max-manual-pdb 123 100.0
sas_results goal_count 87 89.66
sas_results max 58 100.0
sas_results lm_cost 87 100.0
sas_results ff 123 79.67
sas_results causal_graph 77 75.32
test_results_12 blind 78 100.0
test_results_12 merge-and-shrink 114 100.0
test_results_12 max-systematic-pdb 88 100.0
test_results_12 context_enhanced_additive 99 61.62
test_results_12 max-manual-pdb 95 100.0
test_results_12 goal_count 103 99.03
test_results_12 m

In [45]:
dc_results_dict = {folder: {method: glob.glob( os.path.join(folder, method) + '/problem*.txt' ) 
                         for method in os.listdir( os.path.join(folder) ) if 'DS_Store' not in method } 
                         for folder in os.listdir('./') if ('DeepCubeA' in folder) and os.path.isdir(folder)}

dc_results_dict.keys()

dict_keys(['DeepCubeA_18', 'DeepCubeA'])

In [47]:
for key in dc_results_dict:
    for subkey in dc_results_dict[key]:
        count = 0
        count2 = 0
        for item in dc_results_dict[key][subkey]:

            if '18' in key:
                if 'clemens' in subkey:
                    key_in_opt_dict = 'clemens_problems_18'
                elif 'test' in subkey:
                    key_in_opt_dict = 'test_problems_18'
            else:
                if 'clemens' in subkey:
                    key_in_opt_dict = 'clemens_problems_18'
                elif 'test' in subkey:
                    key_in_opt_dict = 'test_problems_12'

            item_basename = os.path.basename(item)
            item_name_opt = 'plan_' + item_basename

            opt_item_path = opt_dict[key_in_opt_dict][0].split('plan_problem')[0] + item_name_opt

            if not os.path.exists(opt_item_path):
                # print('File not found', opt_item_path)
                continue
            count2 += 1
            with open(opt_item_path, 'r') as f:
                opt_content = f.read()
            
            with open(item, 'r') as f:
                content = f.read()

            item_cost = int(content.split('\n')[0].split(';;')[1].strip())
            opt_item_cost = re.search(r'(\d+)', opt_content.split('*')[0] ).group(1)
            if '18' not in key and '18' in key_in_opt_dict:
                opt_item_cost = re.search(r'(\d+)', opt_content.split('*')[1] ).group(1)

            if item_cost == int(opt_item_cost):
                count += 1
        print(key, subkey, key_in_opt_dict, len(dc_results_dict[key][subkey]), count2, count, round(count/count2 * 100,2) )
            


DeepCubeA_18 problems_test_out test_problems_18 200 200 135 67.5
DeepCubeA_18 clemens_test_out clemens_problems_18 200 200 15 7.5
DeepCubeA problems_test_out test_problems_12 200 200 189 94.5
DeepCubeA clemens_test_out clemens_problems_18 200 200 157 78.5
